In [33]:
import numpy as np
import random
import sympy 
import scipy.stats as st
import scipy.special
import pandas as pd
import csv
import math
from decimal import *
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
import datetime

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

from tqdm import tqdm
from itertools import repeat

from shapely.geometry import LineString

from matplotlib import cm
from numpy import linspace
import itertools

from copy import deepcopy

In [4]:
R = 36
def station_number(sta, RA_RU,Lmax, K, delta,  TXOP, Imax, print_able, min, max):

    a_j_i=0
    i=0
    k=0
    n=0

    c=0
    Nn=0
    j=0
    Nn_index=0
    a=0 
    Sum=0 # "sum" can't be use, replace with "Sum"
    temp=0
    temp1=0
    access_delay=0
    counter_access_delay=0
    success_sta=0
    success_packet=0
    success_probability=0
    total_cdf=0
    Imax = Imax + 1
    retrans =Lmax
    Ri = [0 for _ in range(0,Imax+1)] 
    M  = [0 for _ in range(0,Imax+1)] 
    M_i_s_C = [0 for _ in range(0,Imax+1)] 
    M_i    = [ [0] * (retrans + 1) for _ in range(0, Imax+1)]
    M_i_s  = [ [0] * (retrans + 1) for _ in range(0, Imax+1)]
    M_i_f  = [ [0] * (retrans + 1) for _ in range(0, Imax+1)]
    OCW = [0 for _ in range(0,retrans+1) ]
    
    # setting of OCWmin/max can be moved to the outside of the function
    OCWmin = min
    OCWmax = max
    # setting OCWn
    OCW[1]=OCWmin
    # setting all OCW value     
    for n in range (2, Lmax+1):
        temp_OCW=2*OCW[n-1]+1
        if temp_OCW<=OCWmax:
            OCW[n]=temp_OCW
        else:
            OCW[n]=OCW[n-1]
    # 
    for i in range (1,Imax+1): #(i=1;i<=Imax;i++):
        Ri[i] = RA_RU
        for n in range (1, Lmax+1):
            
            #/** initial condition (for "a")
            if n == 1 :
                if i==1 and Ri[i]>=OCW[n] :
                    a_j_i = 1
                elif i==1 and Ri[i]<OCW[n] :
                    a_j_i = (Ri[i]+1)/(OCW[n]+1) #remove (float)
                elif i>1 :
                    temp = 0
                    for c in range (1,i): #c=1;c<=i-1;c++){
                        temp += Ri[c] #// Sum the Ri
                    if (OCW[n] - temp) >= Ri[i] : #//
                        a_j_i = (Ri[i]/(OCW[n]+1)) #remove (float)
                    elif (OCW[n] - temp) <= 0 :
                        a_j_i = 0 #// 
                    else :
                        a_j_i = (OCW[n]-temp)/(OCW[n]+1) #remove (float)

                M_i[i][n] = a_j_i * sta

            elif n <= i :             #// change to n<=2 ?
                #/** Mi[n]=sigma (a_j_i*Mj,F[n-1]) */ 
                for k in range (1, i) : #(k=1;k<=(i-1);k++){
                    
                    if k == i-1 and Ri[i] < OCW[n] :
                        a_j_i = (Ri[i]+1)/(OCW[n]+1) # remove (float)
                        
                    elif k == i-1 and Ri[i] >= OCW[n] : #//(i==1 && Ri[i]>=OCW[n]){
                        a_j_i = 1
                        
                    elif k < i-1 :
                        temp1 = 0
                        
                        for c in range (k+1, i) : #(c=k+1;c<=i-1;c++){   
                            temp1 += Ri[c]
                            
                        if OCW[n] - temp1 >= Ri[i] :   
                            a_j_i = (Ri[i]/(OCW[n]+1)) # remove (float)
                            
                        elif OCW[n] - temp1 <= 0 :
                            a_j_i = 0
                            
                        else :
                            a_j_i = (OCW[n]-temp1)/(OCW[n]+1) # remove (float)
                            
                    M_i[i][n] += a_j_i * M_i_f[k][n-1]
                    
            M[i] += M_i[i][n]
            
            #/** statistics */  
        for n in range (1,Lmax+1) : # n=1;n<=Lmax;n++){
            if Ri[i] != 0: #avoid Ri=0
                M_i_s[i][n] = M_i[i][n] * math.exp(-M[i]/Ri[i])  # remove (double)
            M_i_f[i][n] = M_i[i][n] - M_i_s[i][n]       # remove (double)
            M_i_s_C[i] += M_i_s[i][n]


  #/** performance metric */  
  #/** success probability */ 
        success_packet += M_i_s_C[i]*K
        success_sta += M_i_s_C[i]
#         success_probability = success_packet/(sta*K) # remove (double)
        if (i==Imax-1 and print_able==1) :
            success_probability = success_packet/(sta*K) # remove (double)

        counter_access_delay += M_i_s_C[i]* float( K*i + ((K-1)*K)/2)
    
        if (i==Imax-1 and print_able==1) :

            access_delay=(TXOP*counter_access_delay)/(success_sta*K)# remove (double)

    #/** Imax */
    Imax = 0
    for z in range(1,Lmax+1):
        Imax = Imax + math.ceil(OCW[z]/R)
    
    #Throughput 
    through_put= success_packet/Imax

    return Imax,success_probability,access_delay,through_put


In [5]:
# 檢查所有的組合在給定條件下(R,T,M)的表現 (可以一次觀測所有T下不同組合的表現)

# color=iter(cm.rainbow(np.linspace(0,1,len(T_1)+1)))
line=iter(  ['-','--','-.',':']*30)
mark =iter( ['o','^','*','v','s','p','h','<','>','s','p','h','x']*30)


for T in range(list_main[0][3],list_main[-1][3]+1):
    x=0 
    y=0

    for i in range(len(list_main)):
        if list_main[i][3] == T:
            x= x+1
        else:
            continue

    Ps_list = [[] for i in repeat(None, x)]

    for i in range(len(list_main)):
        if list_main[i][3] == T:
            Ps_list[y].append(list_main[i])
            y = y+1
        else:
            continue
            
#     if (len(Ps_list)>0):
#         print(Ps_list)    


    M = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]

    for i in range(len(Ps_list)):
        for n in range(len(M)):
            the_Imax = 0
            the_Ps=0
            the_Da=0
            the_Tp=0
            the_Imax,the_Ps,the_Da,the_Tp = station_number( M[n], R, Ps_list[i][0][0], 1, 0, 5.681,Ps_list[i][0][3],1,Ps_list[i][0][1],Ps_list[i][0][2])
#             print("Ps: ", the_Ps,"\t Da: ", the_Da,"\t Imax: ", the_Imax)
            Ps_list[i].append(the_Ps)

    #print(Ps_list) 
#     if (len(Ps_list)>1):
    if (len(Ps_list)>0 and T == 8):
        plt.plot(M,Ps_list[0][1:], '-',alpha = 0,color = 'purple',label = "[Lmax, OCWmin, OCWmax, Imax]")
        
        color=iter(cm.rainbow(np.linspace(0,1,len(Ps_list)+1)))
        for i in range(len(Ps_list)):
#             plt.plot(M,Ps_list[i][1:], '-',label = "Ana "+ str(Ps_list[i][0]))

            c=next(color)
            l = next(line)
            plt.plot(M,Ps_list[i][1:], '-', color =c,linestyle = l, linewidth = 2,label = "Ana "+ str(Ps_list[i][0]))
#             plt.plot(M_1,averaged_Ps[i][1], '-', color =c,linestyle = l, linewidth = 2,label = "Ana "+', Total  $Ps$' +', when $T2 = $' +str(G2_Ps[i][0])+', $T1 = $'+ str(T1) )            
            
        # Compare with simulation
        m =next(mark)
#         plt.plot(sim_1.M[0:20],sim_1.Ps[0:20], '*',color = 'black',markersize=6,marker = m,label = 'sim ['+str(sim_1.Lmax[0])+", "+str(sim_1.OCWmin[0])+", "+str(sim_1.OCWmax[0])+", "+str(sim_1.Imax[0])+"]")
#         m =next(mark) 
#         plt.plot(sim_1.M[20:40],sim_1.Ps[20:40], '*',color = 'black',markersize=6,marker = m,label = 'sim ['+str(sim_1.Lmax[20])+", "+str(sim_1.OCWmin[20])+", "+str(sim_1.OCWmax[20])+", "+str(sim_1.Imax[20])+"]")
#         m =next(mark) 
#         plt.plot(sim_1.M[40:60],sim_1.Ps[40:60], '*',color = 'black',markersize=6,marker = m,label = 'sim ['+str(sim_1.Lmax[40])+", "+str(sim_1.OCWmin[40])+", "+str(sim_1.OCWmax[40])+", "+str(sim_1.Imax[45])+"]")
#         m =next(mark) 
#         plt.plot(sim_1.M[60:80],sim_1.Ps[60:80], '*',color = 'black',markersize=6,marker = m,label = 'sim ['+str(sim_1.Lmax[60])+", "+str(sim_1.OCWmin[60])+", "+str(sim_1.OCWmax[60])+", "+str(sim_1.Imax[61])+"]")
#         m =next(mark) 
#         plt.plot(sim_1.M[80:100],sim_1.Ps[80:100], '*',color = 'black',markersize=6,marker = m,label = 'sim ['+str(sim_1.Lmax[80])+", "+str(sim_1.OCWmin[80])+", "+str(sim_1.OCWmax[80])+", "+str(sim_1.Imax[81])+"]")


        
               
        
        plt.title('$R = $' + str(R) +', $T(I_{max}) = $' + str(T) , fontsize = 18)
        plt.xlabel("$M$", fontsize = 14)
        plt.ylabel("Ps", fontsize = 14)
        plt.grid(True, which="both")
        plt.legend(loc='center right', bbox_to_anchor=(1.6, 0.7), fontsize = 12)
        plt.axis([10,200,0,1.1]) #x spacing,x max, y spcaing, y max
        plt.rcParams["figure.figsize"] = (8,6)
        plt.xticks(M)
        plt.yticks(fontsize = 14)
        plt.show()

IndexError: list index out of range

In [ ]:
def all_combination(RA_RU,Lmax_list, min_list, max_list):
    OCWmin =  min_list #[7,15,31,63,127]
    OCWmax =  max_list #[7,15,31,63,127]
    Lmax = Lmax_list #[1,2,3,4,5]
    R = RA_RU
    temp_min = 0
    temp_max = 0
    temp_Lmax = 0
    list = []

    for i in Lmax:
        for x in OCWmin:
            for y in OCWmax:
                OCW = [0 for _ in range(0,i+1) ]
                OCW[1] = x
                for n in range (2, i+1):
                    temp_OCW=2*OCW[n-1]+1
                    if temp_OCW<=y:
                        OCW[n]=temp_OCW
                    else:
                        OCW[n]=OCW[n-1]


                if (not((temp_Lmax == i)&(temp_max == OCW[i])&(temp_min == OCW[1]))):
                    T = 0
                    for z in range(1,i+1):
                        T = T + math.ceil(OCW[z]/R)
                    #print("Lmax: ",i,"\tOCWmin: ",OCW[1],"\tOCWmax: ",OCW[i], "\tT/Imax: ", T)
                    list.append([i,OCW[1],OCW[i],T])
                temp_max = OCW[i]
                temp_min = OCW[1]
                temp_Lmax = i
    list.sort(key = lambda s: s[3])
    return list

In [2]:
the_Imax = 0
the_Ps=0
the_Da=0
the_Tp=0
the_Imax,the_Ps,the_Da,the_Tp = station_number( 10, 9, 1, 1, 0, 5.681,1,1,7,7)
print("Ps: ", the_Ps,"\t Da: ", the_Da,"\t Imax: ", the_Imax)

the_Imax = 0
the_Ps=0
the_Da=0
the_Tp=0
the_Imax,the_Ps,the_Da,the_Tp = station_number( 10, 9, 1, 1, 0, 5.681,2,1,15,15)
print("Ps: ", the_Ps,"\t Da: ", the_Da,"\t Imax: ", the_Imax)

the_Imax = 0
the_Ps=0
the_Da=0
the_Tp=0
the_Imax,the_Ps,the_Da,the_Tp = station_number( 10, 9, 1, 1, 0, 5.681,1,1,15,15)
print("Ps: ", the_Ps,"\t Da: ", the_Da,"\t Imax: ", the_Imax)

NameError: name 'station_number' is not defined

In [3]:
# 給定R來看有哪些組合
R = 37
list_main = []
list_main = all_combination(R,[1,2,3,4,5], [7,15,31,63,127], [7,15,31,63,127])
print("[Lmax, OCWmin, OCWmax, T/Imax]")
for i in range(len(list_main)):
    print(list_main[i])

NameError: name 'all_combination' is not defined